# **LTI&trade; Advantage** bootcamp notebook

## Introduction

The notebook shows how to interact with the LTI Advantage ecosystem from a tool implementer viewpoint. It interacts with an actual test server which has been built as a platform simulator to support this notebook.


## Setup

Here we just import the libraries that will be needed in this notebook, define some utility functions and constants.

In [111]:
import requests
import json
import jwt
import base64
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from IPython.display import display, Markdown

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# for concise code, return full claim prefixed by ims
def fc(claim):
    return "http://imsglobal.org/lti/{0}".format(claim)

def md(mdt):
    display(Markdown(mdt))
    
md_buffer = ''

def mdb(mdt=None):
    global md_buffer
    if mdt:
        md_buffer = md_buffer + '\n' + mdt
    else:
        md(md_buffer)
        md_buffer = ''

# This notebook queries an actual test server. It needs its location:
platform_url='http://localhost:5000'


First we need to get a new tool deployment from the server for this notebook instance to use.
Each tool must have a client_id and a private key that will be used to interact with the platform services and send messages back to the platform.
It also needs the keyset URL that exposes the platform public keys needed to validate the incoming messages.

While this information is required for each tool, how it is obtained by the tool is NOT currently part of the LTI specifications.

In [86]:
r = requests.get(platform_url + '/newtool')
tool_info = json.loads(r.text)

md('### Tool information')
md('Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.')
md('```json\n'+ json.dumps(tool_info, indent=4)+'```')

### Tool information

Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.

```json
{
    "client_id": "0",
    "deployment_id": "deployment_0",
    "key": {
        "e": "AQAB",
        "kty": "RSA",
        "use": "sig",
        "alg": "RS256",
        "d": "SEF6NITJnDCtlxlk_I94QUwgbVvuf_iJGBXKUSz6e8mGnf4GzAp7n0EYKq7s9SPizQUZowBd2zNhzEa8XBfxH1wsEuXvmDcsPI9Y9MX2wtV9ET1eztmOJM_Kc55hmMqd-DcHiTeCpX7jfonal3Lme72hZyAKurRN0YpLr1x8DY0zAPkWYMUyGGX6a3xLyd6Vd5auH5m-c635GumXSGoO5A1b66SdxL17a4XUqwkB92LVZ12cA8YPvuodM2fhS-Qz5eROR2c4ERwhrp40GAuCLKsiHLQR1LXNo-gX1--7_TF_T5mcxc6kW2EU1PtrS0uEDmlSogiHFxlvOniEVYxCmQ=="
    }
}```

### Getting the public keys from the platform

In order to validate the various messages we will receive from the platform, we need to get the key sets. We'll also transform the keys to the `PEM` format that is used by the `jwt` module to decode the messages.

The test platform exposes its keys in a keyset format at a well-know location (.well-known/jwks.json). Other platform might just communicate the keyset url as part of the tool deployment information.

In [87]:
keyset = json.loads(requests.get(platform_url + '/.well-known/jwks.json').text)

md('#### Platform keyset')
md('```json\n'+ json.dumps(keyset, indent=4)+'```')

platform_keys = {}

# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    


#### Platform keyset

```json
{
    "keys": [
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "s6_sJakabBfYiqD-2guvEG5PE71NMlVGuVlyX75etf5CijgWKTtKVQ_62wIL_FW8E-Oo_plgTOm7P7mjdLAkbuOK38YQetrWYrIeNWlO9pCYwbPlVqXDKVA3bWwxoMsYsJwiazm7XutZuHObvptgCPXcg_N3b_htSzxaLVIx0wBF6IWomiCN3_2FTnzS-dNXyyGrmL12HuIpom3MT6YtEREO34qIWUy8O9BQCi3zMQCPkQJ3XKpdcUYMFiuy0FAS6YXyPEdW1pnFGNHCykz0WjUHknfpYz6E-YXrPKjApA92o_PDcv-6CXChFf0lh-I1BAlLqP4jEkFC0S4q0bAgRQ==",
            "kid": "1517438293_0"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "lqKroXxJWaTSuFL54Du72YvuVNAX5kYdAuRj391u2S04I_7uX6ooQuuqEAWdERf3d2OVbWGQ-bfqIeof8FpzXObk8dRVdQJ7ce5Q2g94nhJIRIjtbruudc3G0ioKHNOWkm3RPuKhuVX7qBa0POV5TkVwGDK9vZxiSP6CyKaKnGnx4nba1s5mrZy9B9KmHwXzjKQ3-1aG5jKkmV4djMaQLrxENfpMiJrD9Ne1dkd5Y1OhSC1P_WMFR22cRdA7gfdVLI9OjY-ZYVTY-dqU5iV2FPH3T2b4beeAgO_OM3tld-94tXjforDhl71m7vG5xqiDE3tDAYbaSHYaucc6DOqaEQ==",
            "kid": "1517438293_1"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "wv2udBAVVsgJOGxkzEvck0aAkNF_XOha4gk6DSBr5iXVTON7W9x_r5NEKAgxp6ll06-wi7wRzq60iJwhlAUub1qLUSBIxe_2arBi62kqNPUN8wbbEZkfe_Y6U5IJ3ncQFxDYeI5XSe_qv6b8UT444uxTYzXT3YKnNuRmfkYoe2QUEImthwSDMojIioRDh10CxNMZspgYM0gP8_eWnTbC1mitZU5nooY_FhEgKkcC_dFQSAyGsXE1OGYOuYnPgsS6Km2hcUixyy3eX0Z37tEReZYMC_dklZw2P619xudyRbne0mXd0EbU9F9VEf8tySVjGiaFWUqA5pOylYDNPZzK_w==",
            "kid": "1517438293_2"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "oGLInx8xzUWzZm8Xos5ZMZurRjQXplRCVmEpou9c4KiKKGQ9c0kdkdR2QZyYUTqVPgcjIEs2-YydHysuKNjHcKsK05v09OvstObrKtb3eTcnyZbtbIA9yNy57aLZnsIdkvwV4RpAkVOAwQHruDHKEjdbGxmredB9OyHHfuLXcNdAuuZQPqP772oPe9ErE2IZ_W2_uaCcNlh8657TmLHUXVnvFoaq2ev8vsS2wJ5eax54jIKhLA2GJS-vlPMJjdi2H3r-KgkXZPq2tWGzwKm-JeTvwF50OfFam5zmoKN9dBoCm5097KoP5vCqd6JmN69RgBhNCDgSaf_qiPAS8Xsf_w==",
            "kid": "1517438293_3"
        }
    ]
}```

## Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

[Deep Linking](https://www.imsglobal.org/specs/lticiv1p0) is a **UI flow**, it is an important piece that is sometimes missed on 1st reading. The user is redirected from the platform to the tool to pick or create one or multiple piece of content (often, LTI links), and the the tool redirects the UI back to the tool with the actual selection (or an empty payload if nothing was picked or created).

So there are 2 messages:

1. `ContentItemRequestSelection` from the platform to the tool to start the picking/create session. This is a typical platform launch that contains the context and the user information, and what kind of content items may be created in this flow (for example, this flow might indicate it only wants LTI links).
1. `ContentItemSelection` from the tool back to the plaform using the `content_item_return_url` provided in the request.

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request.

![deep linking](assets/lti_advantage_deeplinking.png "LTI Deep Link")

### Setup: Getting a Deep Linking Request

This notebook is not a tool actually launched by the platform, so the test platform as way to give us the token that it would include in an actual HTTP POSTrequest, so we can build a mock POST request including the parameter `post_data`.

In [91]:
r = requests.get("{}/tool/{}/cisr".format(platform_url, tool_info['client_id']))

post_data = {
    'id_token':r.text
}

### Task 1: Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.


In [92]:
# Let's get the kid so we can get the proper public key
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

md('#### Message properly signed! Decoded ContentItemSelectionRequest message:')
md('```json\n'+ json.dumps(content_item_message, indent=4)+'```')


#### Message properly signed! Decoded ContentItemSelectionRequest message:

```json
{
    "iss": "http://localhost:5000",
    "http://imsglobal.org/lti/version": "1.3.0",
    "sub": "LTIBCU_21",
    "iat": 1517438408,
    "aud": "0",
    "family_name": "Yevheniya",
    "http://imsglobal.org/lti/deployment_id": "deployment_0",
    "http://imsglobal.org/lti/context": {
        "title": "LTI Bootcamp Course",
        "type": [
            "CourseSection"
        ],
        "id": "78326e10-06d7-11e8-a22c-c48e8ffb7857",
        "label": "LTI Bootcamp Course"
    },
    "exp": 1517438468,
    "nonce": "b0731040-06d7-11e8-a22c-c48e8ffb7857",
    "role": [
        "http://purl.imsglobal.org/vocab/lis/v2/membership#Instructor"
    ],
    "email": "Kyou.Yevheniya@example.com",
    "http://imsglobal.org/lti/message_type": "ContentItemSelectionRequest",
    "http://imsglobal.org/lti/tool_platform": {
        "guid": "ltibc_at_1517438294",
        "name": "LTI Bootcamp Platform"
    },
    "given_name": "Kyou",
    "name": "Kyou Yevheniya"
}```

### Task 2: extract the information to decide which experience to show

If this is the first launch for the user or the course, as a tool you may prompt the user for setup information, including account linking or course setup. Ultimately the user will see the authoring or picking interface that will allow her to create or select the content items to be added to the course.

Some key attributes of the `ContentItemSelectionRequest` will drive the experience:

In [112]:
mdb('1. What is the current course id? {0}'.format(content_item_message[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(content_item_message['sub']))
mdb()
md('1. Is this user an instructor? {0}'.format(content_item['sub']))
md('1. What kind of content item can be created? {0}'.format(content_item[c('')]))
md('1. Can I return more than one items to be added? {0}'.format(content_item[c('')]))
md('1. Can I return more than one items to be added? {0}'.format(content_item[c('')]))
md('1. Where should I redirect the browser too when done? {0}'.format(content_item[c('')]))
md('1. Is there any data I must pass back to platform when I return? {0}'.format(content_item[c('')]))


1. What is the current course id? 78326e10-06d7-11e8-a22c-c48e8ffb7857
1. What is the current user id? LTIBCU_21

NameError: name 'content_item' is not defined

### Task 3: redirect the user back to the platform with the content item selection

After the end of the interaction, so user is sent back to the platform throught a browser redirection using an HTTP POST containing the JWT `ContentItemResponse` message. In this case, we will return 2 LTI links, one being graded.

In [ ]:
# Let's start by adding the JWS security claims
content_item_response = {
    'iss': tool_info.client_id,
    'aud': content_item_message['iss']
}